In [1]:
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
#Adding Column Names
cols = ['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']  

In [3]:
#Reading Train Data
data_train = pd.read_csv("/dbfs/FileStore/tables/iris_training.csv", skiprows=[0], names=cols) 

In [4]:
data_train.head()


Out[12]: 
 sepal_len sepal_wid petal_len petal_wid class
0 6.4 2.8 5.6 2.2 2
1 5.0 2.3 3.3 1.0 1
2 4.9 2.5 4.5 1.7 2
3 4.9 3.1 1.5 0.1 0
4 5.7 3.8 1.7 0.3 0

In [5]:
data_train.shape

Out[13]: (120, 5)

In [6]:
#Hot encoding the target column so that we can support multi classes.
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(np.array(data_train.iloc[:, -1]).reshape(-1,1))





In [7]:
# Create xtrain data.
x_train = data_train.iloc[:, :-1]
x_train.head()

Out[15]: 
 sepal_len sepal_wid petal_len petal_wid
0 6.4 2.8 5.6 2.2
1 5.0 2.3 3.3 1.0
2 4.9 2.5 4.5 1.7
3 4.9 3.1 1.5 0.1
4 5.7 3.8 1.7 0.3

In [8]:
x_train.shape

Out[16]: (120, 4)

In [9]:
#Using the encoded columns as Y train
y_train = onehot_encoded


In [10]:
y_train.shape

Out[18]: (120, 3)

In [11]:
#Reading the test data.
data_test = pd.read_csv("/dbfs/FileStore/tables/iris_test.csv",header=None,skiprows=[0],names=cols)

In [12]:
data_test.head()
data_test.shape

Out[20]: (30, 5)

In [13]:
#Creating x_test
x_test = data_test.iloc[:, :-1]

In [14]:
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(np.array(data_test.iloc[:, -1]).reshape(-1,1))


In [15]:
#Using hotencoding to create Y cols
y_test = onehot_encoded

In [16]:
y_test.shape

In [17]:
# Create the model
#X has 4 featues, weights= 4 * 3
# biases are 3 since we are expecting 3 class output.
x = tf.placeholder(tf.float32, [None, 4])
W = tf.Variable(tf.zeros([4, 3]))
b = tf.Variable(tf.zeros([3]))
y = tf.matmul(x, W) + b


In [18]:
# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 3])
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

WARNING:tensorflow:From <command-1173529014475451>:3: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

In [19]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [20]:
# Train
for i in range(1000):
    batch_xs, batch_ys = x_train, y_train
    sess.run(train_step, {x: batch_xs, y_: batch_ys})
    
   
# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('accuracy = %10.4f' % sess.run(accuracy, feed_dict={x: x_test,
                                  y_:y_test}))
print('loss = %10.4f' % sess.run(cross_entropy,feed_dict={x: x_test,
                                  y_:y_test}))


accuracy = 0.9667
loss = 0.0546